In [ ]:
from langchain.schema import AIMessage
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from fpdf import FPDF
import random
import pdfplumber
import warnings
import dotenv

warnings.filterwarnings("ignore")
dotenv.load_dotenv()

In [ ]:
class State(TypedDict):
    # 상태를 정의해주세요.
    # query, answer, messages, tool_call을 가지고 있습니다.


In [ ]:
llm = # 모델을 호출해주세요.

In [ ]:
@tool
def read_pdf(file_path: str) -> str:
    """
    PDF 파일에서 텍스트를 추출하는 도구입니다.
    표 형식 또는 일반 텍스트가 포함된 PDF를 읽고 문자열로 반환합니다.
    
    file_path 예시: './reports/report.pdf'
    """
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip() if text.strip() else "❌ PDF에서 텍스트를 추출할 수 없습니다."
    except Exception as e:
        return f"❌ PDF 읽기 오류: {str(e)}"

In [ ]:
@tool
def write_pdf(content: str, filename: str = "output.pdf"):
    """
    텍스트를 PDF 파일로 저장하는 도구입니다.
    PDF형태의 문서로 만들어야할 때 이 도구를 사용하세요.
    """
    
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    font_path = "C:\Windows\Fonts\MALGUN.TTF"  # <-- 여기에 실제 폰트 파일이 있어야 함

    try:
        pdf.add_font("malgun", "", font_path, uni=True)
        pdf.set_font("malgun", size=12)
    except:
        raise ValueError("한글 폰트가 존재하지 않습니다.")
    
    for line in content.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(f"./reports/{filename}")

    print(f"PDF saved as ./reports/{filename}")

    return {"content":content, "filename":filename}

In [ ]:
# TavilySearchResults : 웹 검색 도구
# PythonAstREPLTool : 파이썬 코드 실행 도구
# write_pdf : pdf 생성 도구
# read_pdf : pdf 읽기 도구
# file_delete : 파일 삭제 도구
# list_directory : 파일 목록 읽기 도구

tools = [TavilySearchResults(include_domains=["naver.com", "google.com"]), PythonAstREPLTool(), write_pdf, read_pdf, *FileManagementToolkit(root_dir="./reports/",
                                                                            selected_tools=["file_delete","list_directory"]).get_tools()]
search_tool, code_tool, write_tool, read_tool, delete_tool, listdir_tool= tools

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# PDF 쓰기 도구 예시

write_tool.invoke({"content":"안녕하세요? \n세미나에 참석해주셔서 감사합니다.", "filename":"Write_pdf_test.pdf", "summary":False})

In [ ]:
# PDF 읽기 도구 예시

print("\n\n", read_tool.invoke("./reports/Write_pdf_test.pdf"))

In [ ]:
# 파일 목록 도구 예시

print(listdir_tool.invoke(""))

In [ ]:
# 삭제 도구 예시

delete_tool.invoke("Write_pdf_test.pdf")

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def shorterm_memory(state:State):

    history_length = # 히스토리의 길이를 임의로 설정해주세요.

    if len(state["messages"]) > history_length:
        history = state["messages"][-history_length:-1]
    elif len(state["messages"]) == 1:
        history = ""
    else:
        history = state["messages"][:-1]

    return history

In [ ]:
class HistoryChecker(BaseModel):
    """
    닥스트링을 작성해주세요.
    """

    yes_no : Literal["yes", "no"] = Field(..., description=
                                          # """description을 작성해주세요.""")

In [ ]:
# history_checker를 정의해주세요.

history_checker = #

In [ ]:
def history_check(state:State):
    """
    기억으로 답을 할 수 있는지 체크하는 분기 역할
    프롬프트는 history와 query를 입력으로 받습니다.
    """


    prompt = PromptTemplate.from_template("""

                # 프롬프트를 작성하세요.
                                          
                """)
    
    chain = # 체인을 생성하세요.

    history = shorterm_memory(state)

    result = # 체인을 실행시키세요.

    return result.yes_no

In [ ]:
def memory_chat(state:State):
    """
    history_check의 응답이 "yes"일시 도달하는 노드
    """

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.
                
                대화 기록 : {history}
                                          
                질문 : {query}
                                          
                """)

    
    chain = prompt | llm

    history = shorterm_memory(state)

    answer = chain.invoke({"history":history,
                           "query":state["query"]})
    
    if len(state["tool_call"]) == 0:
        return {"answer": # answer의 반환값을 입력해주세요.
                "messages": # messages의 반환값을 입력해주세요.
                "tool_call":"사용된 기록 없음."}
    else:
        return {"answer":answer.content,
                "messages":answer}

In [ ]:
def history_node(state:State):
    if len(state["messages"]) == 1:
        return {"answer":"답변 없음",
                "tool_call":"사용된 도구 없음"} 
    else:
        return state

In [ ]:
def select(
    state: State,
):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.
                최근에 사용한 도구가 있다면 도구도 참고하세요. 다른 도구를 사용하는 것이 더 좋은 방법이 될 수 있습니다.
                                                    
                대화 기록 : {history}     
                                          
                최근 사용한 도구 : {tool_name}
                                        
                정답 : {answer}
                                        
                질문 : {query}
                                          
                """)

    chain = prompt | llm_with_tools

    history = shorterm_memory(state)

    result = chain.invoke({"history" : history,
                           "tool_name" : # 상태의 tool_call을 입력해주세요.,
                            "answer": # 상태의 answer를 입력해주세요.,
                            "query": # 상태의 query를 입력해주세요.})

    if hasattr(result, "tool_calls") and len(result.tool_calls) > 0:
        tool_calls = result.tool_calls
        return {"messages": result,
                "tool_call": [result]}
    else:
        return {"messages":AIMessage(content=f"""도구를 선택하지 못했습니다. 적절한 도구를 재선택하세요.
                                        """),
                                    "tool_call":None}

    

In [ ]:
# 도구 실행 노드 생성

tool_node = ToolNode(tools)

In [ ]:
class AnswerChecker(BaseModel):
    """
    닥스트링을 입력해주세요.
    """


    end : Literal["end", "tool"] = Field(..., description="""
                                                        description을 입력하세요.
                                                        """)    

In [ ]:
# answer_checker를 정의해주세요.

answer_checker = #

In [ ]:
def answer_check(state:State):

    prompt = PromptTemplate.from_template("""
    당신은 정답 분류기 어시스턴트트입니다.
    
    정답이 질문을 해결할 수 있는지 여부를 판단합니다.
    질문을 해결할 수 없다면 도구를 이용합니다.

    질문을 해결할 수 있다면 "end", 아니라면 "tool"을 반환합니다.
                                          
    기존 History도 참고하여 답변하세요.
                                          
    History : {history}
                            
    정답 : {answer}
                            
    질문 : {query}
    """)

    chain = prompt | answer_checker

    history = # 히스토리를 가져오세요.

    result = # 체인을 실행시키세요.
    
    return # 반환값을 설정하세요.

In [ ]:
def response(state:State):

    return {"answer":state["messages"][-1]}

In [ ]:
# graph_builder를 정의하세요.

graph_builder = #

In [ ]:
# 노드와 엣지를 구성하세요.

In [ ]:
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# 랜덤 config 정의 함수

def reset_config(limit=15):
 
    thread_id=random.randint(1, 999999)

    config = RunnableConfig(recursion_limit=limit, configurable={"thread_id": thread_id})

    return config

In [ ]:
config = reset_config()

In [ ]:
graph

In [ ]:
# 답변 함수

def streaming(query, config):

    result = graph.stream({"messages":("user", query),
                         "query":query}, config=config)
    for step in result:
        for k, v in step.items():
            print(f"\n\n=== {k} ===\n\n")
            try:
                print(v)
            except:
                pass
    
    return 

In [ ]:
# 자유롭게 에이전트에게 물어보세요.